In [ ]:
https://adk-default-service-name-572950975329.us-central1.run.app

# list the agent

In [2]:
import requests
import json
# 2️⃣ GET request
url_get = "https://adk-default-service-name-572950975329.us-central1.run.app/list-apps"
response_get = requests.get(url_get)

print("GET response:", response_get.status_code, response_get.text)


GET response: 200 ["orchestrator"]


# check open sessions

In [ ]:
import requests
from pprint import pprint

base = "https://adk-default-service-name-572950975329.us-central1.run.app"
url  = f"{base}/apps/orchestrator/users/steven/sessions"

r = requests.get(url)
r.raise_for_status()
payload = r.json()

def iter_sessions(payload):
    """Yield session dicts from various payload shapes."""
    if isinstance(payload, list):
        for s in payload:
            if isinstance(s, dict):
                yield s
    elif isinstance(payload, dict):
        # common containers
        for key in ("sessions", "items", "data", "results"):
            if key in payload and isinstance(payload[key], list):
                for s in payload[key]:
                    if isinstance(s, dict):
                        yield s
                return
        # map of id -> session
        for sid, s in payload.items():
            if isinstance(s, dict):
                s.setdefault("id", sid)
                yield s

def get_state_str(s):
    """Extract a state/status/phase as an UPPER string if present."""
    val = s.get("state") or s.get("status") or s.get("phase")
    if isinstance(val, str):
        return val.upper()
    if isinstance(val, dict):
        # try common fields within a nested object
        for k in ("name", "value", "label", "state", "status", "phase"):
            if isinstance(val.get(k), str):
                return val[k].upper()
    # boolean flags
    if s.get("open") is True or s.get("is_active") is True:
        return "ACTIVE"
    return ""

def get_end_time(s):
    """Unify possible end/close time fields."""
    return s.get("ended_at") or s.get("end_time") or s.get("closed_at")

sessions = list(iter_sessions(payload))

open_like = {"OPEN", "ACTIVE", "RUNNING"}
open_sessions = [s for s in sessions if (get_state_str(s) in open_like) or not get_end_time(s)]

print("All sessions:")
pprint(sessions)
print("========================")
print("\nOpen sessions:")
pprint(open_sessions)


All sessions:
[{'appName': 'orchestrator',
  'events': [],
  'id': 'sess-1',
  'lastUpdateTime': 1762578404.118867,
  'state': {'init': 'ok'},
  'userId': 'steven'}]

Open sessions:
[{'appName': 'orchestrator',
  'events': [],
  'id': 'sess-1',
  'lastUpdateTime': 1762578404.118867,
  'state': {'init': 'ok'},
  'userId': 'steven'}]


# IF NEEDED create open sesssion

In [5]:
import requests
import json

# 1️⃣ POST request (with JSON payload)
url_post = "https://adk-default-service-name-572950975329.us-central1.run.app/apps/orchestrator/users/steven/sessions/sess-1"
headers = {"Content-Type": "application/json"}
data = {"init": "ok"}

response_post = requests.post(url_post, headers=headers, json=data)
print("POST response:", response_post.status_code, response_post.text)


POST response: 409 {"detail":"Session already exists: sess-1"}


# talk to the agent

In [13]:
import time
import requests
from pprint import pprint

BASE, APP, USER, SID = (
    "https://adk-default-service-name-572950975329.us-central1.run.app",
    "orchestrator",
    "steven",
    "sess-1",
)
HEADERS = {"Content-Type": "application/json"}
# HEADERS["Authorization"] = "Bearer <TOKEN>"  # if your Cloud Run needs it

def send_user_message(text: str):
    url = f"{BASE}/apps/{APP}/users/{USER}/sessions/{SID}"
    payload = {
        "stateDelta": {
            "events": [
                {"role": "user", "parts": [{"text": text}]}
            ]
        }
    }
    r = requests.patch(url, headers=HEADERS, json=payload, timeout=60)
    r.raise_for_status()
    return r.json() if r.headers.get("content-type","").startswith("application/json") else r.text

def fetch_events():
    url = f"{BASE}/apps/{APP}/users/{USER}/sessions/{SID}/events"
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_session():
    url = f"{BASE}/apps/{APP}/users/{USER}/sessions/{SID}"
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

def extract_assistant_text(events_or_state):
    """
    Accepts either:
      - the list returned by GET /events, or
      - a full session dict (we'll look into ['state']['events'] if present).
    Returns a list of assistant strings (latest last).
    """
    msgs = []

    # 1) If it's a session dict, try state.events first
    if isinstance(events_or_state, dict):
        state_events = events_or_state.get("state", {}).get("events", [])
        for ev in state_events:
            if ev.get("role") == "assistant":
                # Gemini-style: parts -> [{text: "..."}]
                parts = ev.get("parts") or []
                for p in parts:
                    if isinstance(p, dict) and "text" in p:
                        msgs.append(p["text"])
        # Some ADK variants also mirror assistant lines in top-level 'events'
        events_list = events_or_state.get("events", [])
    else:
        events_list = events_or_state

    # 2) Walk the event timeline for assistant author
    for ev in events_list or []:
        # direct author field
        if ev.get("author") == "assistant":
            # Sometimes assistant text is in actions.stateDelta.events[*].parts[*].text
            actions = ev.get("actions", {})
            sd = actions.get("stateDelta", {}) if isinstance(actions, dict) else {}
            for m in sd.get("events", []):
                if m.get("role") == "assistant":
                    for p in m.get("parts", []) or []:
                        if isinstance(p, dict) and "text" in p:
                            msgs.append(p["text"])
        # or a normalized event
        if ev.get("role") == "assistant":
            for p in (ev.get("parts") or []):
                if isinstance(p, dict) and "text" in p:
                    msgs.append(p["text"])

    return msgs

# ---- 1) Send a message
_ = send_user_message("Summarize today's plan.")

# ---- 2) Poll for assistant reply (up to ~15s)
deadline = time.time() + 15
assistant_lines = []
while time.time() < deadline and not assistant_lines:
    try:
        # Prefer /events (often fresher); fall back to full session
        timeline = fetch_events()
        assistant_lines = extract_assistant_text(timeline)
        if not assistant_lines:
            sess = fetch_session()
            assistant_lines = extract_assistant_text(sess)
    except Exception:
        pass
    if not assistant_lines:
        time.sleep(1.0)

if assistant_lines:
    print("Assistant reply:")
    print(assistant_lines[-1])  # latest line
else:
    print("No assistant message found yet. Here is the latest session snapshot:")
    pprint(fetch_session())


No assistant message found yet. Here is the latest session snapshot:
{'appName': 'orchestrator',
 'events': [{'actions': {'artifactDelta': {},
                         'requestedAuthConfigs': {},
                         'requestedToolConfirmations': {},
                         'stateDelta': {'events': [{'parts': [{'text': 'Summarize '
                                                                       "today's "
                                                                       'plan.'}],
                                                    'role': 'user'}]}},
             'author': 'user',
             'id': '365618a6-15e7-42c7-8153-3b0290ac142b',
             'invocationId': 'p-c3fa3134-2493-495f-80bc-d75f1907603a',
             'timestamp': 1762579290.586646},
            {'actions': {'artifactDelta': {},
                         'requestedAuthConfigs': {},
                         'requestedToolConfirmations': {},
                         'stateDelta': {'events': [{'parts':